In [1]:
import numpy as np
import cantera as ct

In [ ]:
# physical constant
e = 1.60217657e-19 # C
k_b = 1.3806488e-23 # J/K
N_a = 6.02214129e23 # /mol
epsilon_0 = 8.854187817e-12 # vecuum permittivity F/m

# parameters for transport model
K1 = 1.767
K2 = 0.72
kappa = 0.095

# paramerters of simulation
num_species = 53
E = 100

# variables of chemical species
W = np.zeros(num_species) # molar mass kg/mol
q = np.zeros(num_species)
m = W/N_a # molecular mass 

# state value of simulation
T = 1000 # K
X = np.zeros(num_species)

# dependent variable
n = 1e3 # number density /m3
N = 1e3 # background density /m3
W_bar = 1e5 # kg/mol

# polarizavility m3
alpha = np.zeros(num_species)
# load parameter from table

# ionization energy J
I_e = np.zeros(num_species) 
# load parameter from table

# (19)
C6_i = np.zeros(num_species)
for i in range(num_species):
    C6_i[i] = 3*np.pi*epsilon_0*alpha[i]**2*I_e[i]

# (17) ratio of dispersion to induction forces -
xi = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        xi[i, j] = alpha[i]/(q[i]**2*(1 + (2*alpha[i]/alpha[j])**(2/3))*alpha[j]**0.5)

# (18) cross section diameter m
sigma = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        sigma[i, j] = K1*(alpha[i]**(1/3) + alpha[j]**(1/3))/(alpha[i]*alpha[j]*(1 + 1/xi[i,j]))**kappa

# (16) dispersion coefficient C2*m5
C6 = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        C6[i, j] = 2*C6_i[i]*C6_i[j]/((alpha[j]/alpha[i])*C6_i[i] + (alpha[i]/alpha[j])*C6_i[j])

# (15) well depth of the potential function J
epsilon = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        epsilon[i, j] = K2*(alpha[j]*q[i]**2*e**2/(8*np.pi*epsilon_0*sigma[i, j]**4))*(1 + xi[i, j])

# reduced temperature -
T_star = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        T_star[i, j] = k_b*T/epsilon[i, j]

# (14)
gamma = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        gamma[i, j] = (2/q[i]**2/(C6[i, j]/e**2) * alpha[q, j])/(alpha[j]*sigma[i, j]**2)

# reduced collision integral -
Omega_11 = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        if T_star < 0.04:
            # (A1) for 0.01 < T_star < 0.04
            Omega_11[i, j] = 2.97 - 12.0*gamma - 0.887*np.log(T_star) + 3.86*gamma**2 - 6.45*gamma*np.log(T_star) \
                - 0.275*np.log(T_star)**2 + 1.20*gamma**2*np.log(T_star) - 1.24*gamma*np.log(T_star)**2 - 0.164*np.log(T_star)**3
        else:
            # (A2) for 0.04 < T_star < 1000
            Omega_11[i, j] = (1.22 - 0.0343*gamma) + (-0.769 + 0.232*gamma)*np.log(T_star) \
                + (0.306 - 0.165*gamma)*np.log(T_star)**2 + (-0.0465 + 0.0388*gamma)*np.log(T_star)**3 \
                + (0.000624 - 0.00285*gamma)*np.log(T_star)**4 + 0.000238*np.log(T_star)**5

# (7) cross section m2
Omega_D = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        Omega_D[i, j] = np.pi*sigma[i, j]*Omega_11[i, j]

# (5) binary mobility m2/V/s
mu_bi = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        mu_bi[i, j] = (3.0/16.0)*(q[i]*e/Omega_D*N)*np.sqrt(2.0*np.pi*N_a/k_b/T/m[i, j])

# (4) binary diffusivity m2/s
D_bi = np.zeros((num_species, num_species))
for i in range(num_species):
    for j in range(num_species):
        D_bi[i, j] = (3.0/16.0)*(1/Omega_D*N)*np.sqrt(2.0*np.pi*N_a*k_b*T/m[i, j])

# (3) movility m2/V/s
mu = np.zeros(num_species)
for i in range(num_species):
    mu[i] = 1.0/(np.sum(np.divide(X/mu[i, :])))

# (2) diffusivity m2/s
D = np.zeros(num_species)
for i in range(num_species):
    D[i] = np.sum(np.multiply(X, W) - X[i]*W[i])/(W_bar*(np.sum(np.divide(X, D[i, :]))) - X[i] - D[i, i])

# (1) number diffusion flux /m2/s
J = np.zeros(num_species)
for i in range(num_species):
    J[i] = -D[i]*Nabla(n[i]) + q[i]/np.abs(q[i])*mu[i]*E*n[i]